In [48]:
import torch 
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable

from dataset.readVideo import DramaDataset
from NGram.tool import NGram, trigrams
from utils.tokenMaker import Lang
import pickle

useCuda = True
Variable = Variable.cuda if useCuda else Variable

AttributeError: type object 'Variable' has no attribute 'cuda'

In [46]:
torch.Tensor.cuda

<method 'cuda' of 'torch._C._TensorBase' objects>

In [3]:
DataDir="/home/ball/Videos/BrokeEN"
datasets = DramaDataset(basedir=DataDir,
                        maxFrame=0,
                        )
import random
def getSample(size):
    imgs = []
    pres = []
    nexs = []
    for i in range(size):
        pre, nex, img = random.choice(datasets)
        imgs.append(img)
        pres.append(pre)
        nexs.append(nex)
    return pres, nexs, imgs

Total Drama: 113


In [4]:
lang = Lang("test", split=" ")

In [5]:
class Gesd(nn.Module):
    def __init__(self, gamma=1, c=1, reduce=True):
        super(Gesd, self).__init__()
        self.gamma = gamma
        self.c = c
        self.reduce = reduce

    def forward(self, f1,f2):
        l2_norm = ((f1-f2) ** 2).sum(dim=1)
        euclidean = 1 / (1 + l2_norm)
        sigmoid  = 1 / (1 + torch.exp(-1 * self.gamma * ((f1*f2).sum(dim=1) + self.c)))
        output = euclidean * sigmoid
        return output

In [6]:
from model.model import EncoderRNN
from utils.tool import getLastOutputs
from utils.tool import padding, flatMutileLength, Timer, Average

In [7]:
class GesdSimilarity(nn.Module):
    def __init__(self, subencoder_setting, gamma=1, c=1):
        super(GesdSimilarity, self).__init__()
        self.preEncoder = EncoderRNN(**subencoder_setting)
        self.nextEncoder = EncoderRNN(**subencoder_setting)
        self.gesd = Gesd(gamma=gamma, c=c)

    def forward(self, s1, s2):
        if(isinstance(s1, tuple)):
            s1Sentence, s1Lengths = s1
        else:
            s1Sentence = s1
            s1Lengths = [s1.size(1)]*s1.size(0)
            
        if(isinstance(s2, tuple)):
            s2Sentence, s2Lengths = s2
        else:
            s2Sentence = s2
            s2Lengths = [s2.size(1)]*s2.size(0)
            
        feature1, _ = self.preEncoder(s1Sentence)
        feature2, _ = self.nextEncoder(s2Sentence)
        
        feature1 = getLastOutputs(feature1, s1Lengths)
        feature2 = getLastOutputs(feature2, s2Lengths)
        
        output = self.gesd(feature1, feature2)
        return output

In [8]:
subencoderOpt = {
    "word_size": 10000,
    "em_size": 512,
    "hidden_size": 512,
    "output_size": 512 
}
model = GesdSimilarity(subencoderOpt)

/home/ball/anaconda3/lib/python3.5/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [9]:
pres, nexs, imgs = getSample(6)

In [10]:
for pre in pres+nexs:
    lang.addSentance(pre)

In [11]:
vectorTransforms = [torch.LongTensor, Variable]
def sentenceToVector(sentences, lang, sos=False, eos=False):
    vectors = []
    for s in sentences:
        vectors.append(lang.sentenceToVector(s, sos=sos, eos=eos))
    vectors = padding(vectors, lang["PAD"], vectorTransforms)
    return vectors

In [14]:
def makeNegSample(pres, nexs, negSize):
    mpres = []
    mnexts = []
    scores = []
    for pre, nex in zip(pres, nexs):
        while True:
            negs = random.sample(nexs, negSize)
            if not nex in negs:
                break
        mpres +=[pre] * (negSize + 1)
        mnexts += [nex] + negs
        scores += [1] + [0] * negSize
    return mpres, mnexts, scores

In [27]:
data = model.preEncoder(vpres[0])[0][:,-1,:]

In [26]:
vpres[0]

tensor([[  4,   5,   6,   7,   0,   0,   0,   0,   0,   0],
        [  4,   5,   6,   7,   0,   0,   0,   0,   0,   0],
        [  4,   5,   6,   7,   0,   0,   0,   0,   0,   0],
        [  8,   9,  10,   4,  11,  12,  13,  14,  15,   0],
        [  8,   9,  10,   4,  11,  12,  13,  14,  15,   0],
        [  8,   9,  10,   4,  11,  12,  13,  14,  15,   0],
        [ 16,  17,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 16,  17,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 16,  17,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 18,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 18,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 18,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 19,  20,  21,  22,  23,  24,  25,  26,   9,  27],
        [ 19,  20,  21,  22,  23,  24,  25,  26,   9,  27],
        [ 19,  20,  21,  22,  23,  24,  25,  26,   9,  27],
        [ 28,  29,  30,  31,  32,  33,  34,   0,   0,   0],
        [ 28,  29,  30,  31,  32,  33,  

In [15]:
p,n,s = makeNegSample(pres, nexs, negSize=2)

In [16]:
vpres = sentenceToVector(p, lang)
vnexts = sentenceToVector(n, lang)
scores = Variable(torch.Tensor(s))

In [29]:
outputs = gesd(data, data)

In [28]:
gesd = Gesd()

In [18]:
criterion = nn.BCELoss()

In [37]:
loss = criterion(outputs, torch.ones(outputs.size(0)))

In [20]:
torch.empty(3).random_(2)

tensor([ 1.,  0.,  1.])

In [36]:
Variable(torch.ones(outputs.size(0)))

tensor([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.])

In [38]:
loss

tensor(0.1253)

In [40]:
from Similarity import trainer

ImportError: cannot import name 'trainer'

In [42]:
from Similarity import 